In [4]:
import pypsa
import pandas as pd
import glob

In [ ]:
# --------- 1) Scenarios  ----------
# base:   grid connection
#         CHP 0.635 MW_el, 0.79 MW_th
#         solar_freefield 0.840 MWp
#         nothing extendable
# 2:      grid connection
#         CHP 0.635 MW_el, 0.79 MW_th
#         solar_freefield 0.840 MWp
#         elec flow storage 0.72MWh, 0.12MW
#         nothing extendable
# 3:      grid connection
#         CHP 0.635 MW_el, 0.79 MW_th
#         solar_freefield 0.840 MWp
#         elec flow storage 0.72MWh, 0.12MW
#         rooftop PV extendable
# 4:      No grid connection
#         CHP extendable
#         solar_freefield 0.840 MWp
#         elec flow storage extendable
#         rooftop PV extendable
# 5:      No grid connection
#         CHP extendable
#         solar_freefield 0.840 MWp
#         elec flow storage extendable
#         rooftop PV extendable
#         wind extendable



SC = 1
pars = {
  1: dict(grid="True", chp_ext="False",  solar_roof="False", flow_storage="False", flow_storage_ext="False", wind="False"), # 1: 840 kWp PV, no battery
  2: dict(grid="True", chp_ext="False",  solar_roof="False", flow_storage="True", flow_storage_ext="False", wind="False"), # 2: +0.72 MWh battery
  3: dict(grid="True", chp_ext="False",  solar_roof="True", flow_storage="True", flow_storage_ext="False", wind="False"), # 3: 2.0 MW PV + 2.5 MWh battery
  4: dict(grid="False", chp_ext="True",  solar_roof="True", flow_storage="True", flow_storage_ext="True", wind="False"), # 4: 3.9 MW PV + 8.9 MWh battery
  5: dict(grid="False", chp_ext="True",  solar_roof="True", flow_storage="True", flow_storage_ext="True", wind="True"), # 5: 1.3 MW PV + 1.3 MW wind + 4.9 MWh battery
}[SC]

wd = "/home/cs/Documents/Cola/Master/MS_5-Semester/AU-Viborg" # working directory

In [ ]:
# Data
ir = 0.07 # interestrate
data = pd.DataFrame({
    "capex": [400, 1000, 1100, 400, 500, 8, 40], 
    "FOM": [10, 12, 14, 14, 10, 0, 0], # fixed O&M
    "opex": [0, 0, 0.05, 0.17, 0, 0, 0.35],
    "lifetime": [35, 35, 27, 20, 20, 20, 0],
}, index=["PVFree", "PVRoof", "onwind", "CHP", "flow_storage", "thermal_storage", "grid"]
)
# add O&M to capex:
data["capex"] = data["capex"]*(ir*(1+ir)**data["lifetime"]/((1+ir)**data["lifetime"]-1)) + data["FOM"]

# Wind timeseries
wind_ts = pd.read_csv(f"{wd}/data/Wind_Vestas-V90-2000_2019.csv", comment="#")
wind_ts = wind_ts.set_index("time")
wind_ts.index = pd.to_datetime(wind_ts.index)
wind_ts = wind_ts.drop(columns=["local_time"])
wind_ts["CF"] = wind_ts["electricity"]/2000 # 2000 rated power

# Solar timeseries
solar_dict = {'SE': 2.003, 'S': 1.056, 'SW': 6.643, 'E': 0.731, 'NW': 2.003, 'N': 1.056, 'NE': 6.643, 'W': 0.731} #MW
solar_ts_dict = {}
for loc in solar_dict:  # loop directly over dictionary keys
    files = glob.glob(f"{wd}/data/Solar_{loc}*.csv")
    if not files:
        continue
    ts = pd.read_csv(files[0], skiprows=10, skipfooter=11, engine="python")
    ts["time"] = pd.to_datetime(ts["time"].str.replace(":", ""), format="%Y%m%d%H%M")
    ts["time"] = ts["time"].dt.floor("h")
    ts = ts.set_index("time")
    # Compute capacity factor using the dictionary
    solar_ts_dict[loc] = ts["P"] / (solar_dict[loc] * 1000)
solar_ts = pd.DataFrame(solar_ts_dict) # Combine all location series into one DataFrame

L_h = pd.read_excel(f"{wd}/data/Synthese_von_Waermelastprofilen_nach_BDEW.xlsx", sheet_name="Ausgabe")
#L_h["Energie in kWh"].sum() # check if it is 626.57 m3 * 10.16 kWh/m3 = 6365.95 kWh as given in the data
L_h = L_h.rename(columns={"Zeitstempel": "time", "Energie in kWh": "energy"})
L_h = L_h.set_index('time')
L_h.index = pd.to_datetime(L_h.index, unit='h')
L_h.index = L_h.index - pd.DateOffset(years=4) #match 2019 timestamp
L_h = L_h.iloc[:-24] # drop the day after newyears

# Power Demand Timeseries
# Based on that there as of now is no CHP (so full elec demand as given by Björn)
L_e = pd.read_excel(f"{wd}/data/N61616_sum_alle_3.xlsx")
L_e = L_e.iloc[:, [4,8]]
L_e = L_e.rename(columns={"Gældende fra": "time", "Sum 61616 ": "energy"}) 
L_e['time'] = pd.to_datetime(L_e['time']).dt.tz_localize("Europe/Copenhagen", ambiguous="NaT", nonexistent="NaT")
L_e['time'] = L_e['time'].dt.tz_convert("UTC")
L_e.index = L_e['time']
# L_e = L_e.drop(columns=["time"])
# L_e.index = L_e.index + pd.DateOffset(hours=1)
# L_e.index = L_e.index - pd.DateOffset(years=3)
# Create a copy of the last row with timestamp shifted by 1 hour
last_row = L_e.iloc[[-1]].copy()
last_row.index = last_row.index + pd.Timedelta(hours=1)
L_e = pd.concat([L_e, last_row])
L_e.index = wind_ts.index


In [ ]:
# Heat Demand Timeseries
# biogas and natural gas lower heating value:
# https://backend.orbit.dtu.dk/ws/portalfiles/portal/240877731/Danish_roadmap_for_a_sustainable_gas_grid_transition_status_and_potential_role_of_thermal_gasification.pdf
# https://ocw.tudelft.nl/wp-content/uploads/Summary_table_with_heating_values_and_CO2_emissions.pdf
# biogas between 16 MJ/m3 and 28 MJ/m3 ~ 23.1 MJ/m3 = 6.42 kWh/m3
# natural gas 36.6 MJ/m3 = 10.16 kWh/m3
# adjustment factor from naturalgas consumption now to biogas in the future:
# gaf = 10.16/6.42 # multiply with gas consumption then multiply gasconsumption in m3 with 6.42 kWh/m3 to get kWh

In [172]:
# # print(L_h)
# print(L_e)
# print(wind_ts)
# #solar_ts
# # len(L_e.index) 
# # len(wind_ts.index)
# # L_e.index == wind_ts.index
L_e.index.equals(wind_ts.index)
# mask = L_e.index != wind_ts.index
# L_e.index[mask]

# print(f"Number of duplicates: {L_e.index.duplicated().sum()}")
# print(L_e[L_e.index.duplicated(keep=False)]) 
solar_ts.columns
data.at["onwind", "capex"]

np.float64(105.76830741956803)

In [ ]:
# --------- 2) Network ----------
n = pypsa.Network()
n.set_snapshots(wind_ts.index)
n.add("Bus","elec"); n.add("Bus","heat"); n.add("Bus","gas")

n.add("Load","L_e", bus="elec", p_set=L_e["energy"])      # kWh
n.add("Load","L_h", bus="heat", p_set=L_h["energy"])

# CHP: gas -> electricity (40%) + heat (50%), fixed 635 kWel
n.add("Link","CHP",
      bus0="gas", bus1="elec", bus2="heat",
      p_nom=1.5875, # 0.635MW_el/0.4
      efficiency=0.40, efficiency2=0.50,
      capital_cost=data.at["CHP", "capex"],
      marginal_cost=data.at["CHP", "opex"])

# Biogas supply price (€/kWh -> €/MWh)
n.add("Carrier","biogas")
n.add(
    "Generator",
    "biogas_supply",
    bus="gas",
    carrier="biogas",
    p_nom=1e6,
    marginal_cost=0)

# Electricity supply (gridconnection)
n.add("Carrier","grid")
n.add("Generator",
    "grid_import",
    bus="elec",
    p_nom_extendable=True,  # Allow optimization to determine peak capacity
    p_nom_max=1e6,  # Upper limit
    marginal_cost=data.at["grid", "opex"],  # Energy charge: 35 ct/kWh
    capital_cost=data.at["grid", "capex"],  # Capacity charge: 40 €/kW peak? annulized (no lifetime)?
    carrier="grid")

# add rooftop solar
for loc in solar_ts.columns:
    n.add(
        "Generator",
        f"solar_{loc}",
        bus="elec",
        carrier="solar",
        p_max_pu=solar_ts[f"{loc}"],
        capital_cost=data.at["PVRoof", "capex"],
        p_nom_extendable=True,
        p_nom_max = solar_dict[loc]
    )
# add freefield solar
n.add(
    "Generator",
    "solar_S_Free",
    bus="elec",
    carrier="solar",
    p_max_pu=solar_ts["S"], 
    capital_cost=data.at["PVFree", "capex"],
    p_nom_extendable=True,
    p_nom_max = 840
)

# add windpower
n.add(
    "Generator",
    "wind",
    bus="elec",
    carrier="wind",
    p_max_pu=wind_ts["CF"],
    capital_cost=data.at["onwind", "capex"],
    p_nom_extendable=True,
    # p_nom_max = 2000 # own estimate = std windturbine
)

n.add("StorageUnit",
    "flow_storage",
    bus="elec",
    p_nom=120,  # Power capacity in kW
    max_hours=720/120,  # = 6 hours (energy/power ratio)
    efficiency_store=0.9,  # Charging efficiency estimate
    efficiency_dispatch=0.9,  # Discharging efficiency estimate
    cyclic_state_of_charge=True,
    capital_cost=data.at["flow_storage", "capex"],  # CAPEX scaled to power capacity
    marginal_cost=0)  # No cost per kWh charged/discharged

# to little info about thermal storage

In [ ]:
# # add flow storage
# n.add("Store",
#     "flow_storage",
#     bus="flow_storage_bus",
#     e_nom=720,  # Energy capacity in kWh
#     # e_nom_extendable=True,  # Allow optimization to size it
#     # e_nom_max=1e6,  # Maximum capacity
#     capital_cost=data.at["flow_storage", "capex"], 
#     marginal_cost=0,  # No cost per kWh charged/discharged
#     e_cyclic=True,  # Storage level at end = level at start
#     carrier="flow_battery")

# # Add charger (power capacity for charging)
# n.add("Link",
#     "redox_flow_charger",
#     bus0="elec",  # Electricity bus
#     bus1="flow_storage_bus",  # Storage bus
#     p_nom=120,  # Power capacity in kW
#     efficiency=0.95,  # Adjust based on actual round-trip efficiency
#     carrier="battery_charger")

# # Add discharger (power capacity for discharging)
# n.add("Link",
#     "redox_flow_discharger",
#     bus0="redox_flow_storage",  # Storage bus
#     bus1="elec",  # Electricity bus
#     p_nom=120,  # Power capacity in kW
#     efficiency=0.95,  # Adjust based on actual round-trip efficiency
#     carrier="battery_discharger")

In [ ]:
# Interest rate
ir = 0.07

# Raw cost data
data = pd.DataFrame({
    "capex": [400e3, 1000e3, 1100e3, 400e3, 500e3, 8, 0],  # eur/MWp eur/L
    "FOM": [10e3, 12e3, 14e3, 14e3, 10e3, 0, 0],  # Fixed O&M (€/MWp/year)
    "opex": [0, 0, 0.05e3, 0.17e3, 0, 0, 0.35e3],  # Variable O&M (€/MWh)
    "lifetime": [35, 35, 27, 20, 20, 20, 1],  # Grid lifetime=1 to avoid div/0
}, index=["PVFree", "PVRoof", "onwind", "CHP", "flow_storage", "thermal_storage", "grid"])

# Annualize CAPEX and add FOM
# For technologies with capital investment: annualized_capex = CAPEX * CRF + FOM
# CRF (Capital Recovery Factor) = ir*(1+ir)^n / ((1+ir)^n - 1)
data["capex_annual"] = data["capex"] * (ir * (1 + ir)**data["lifetime"] / 
                                         ((1 + ir)**data["lifetime"] - 1)) + data["FOM"]

# Grid costs need special handling
# Since PyPSA optimizes per year, we treat the 40 €/kW as an annual capacity cost
grid_capacity_cost = 40e3  # €/MW annual capacity charge
grid_energy_cost = 0.35e3  # €/MWh energy charge

# --------- Setup Network ----------
n = pypsa.Network()
n.set_snapshots(wind_ts.index)

# Buses
n.add("Bus", "elec")
n.add("Bus", "heat")
n.add("Bus", "gas")

# Loads (use your actual load data)
n.add("Load", "L_e", bus="elec", p_set=L_e["energy"])
n.add("Load", "L_h", bus="heat", p_set=L_h["energy"])

# --------- Generators ----------
# Biogas supply
n.add("Carrier", "biogas")
n.add("Generator",
      "biogas_supply",
      bus="gas",
      carrier="biogas",
      p_nom=1e6,
      marginal_cost=0)  # Cost is in CHP operation

# Grid import
n.add("Carrier", "grid")
n.add("Generator",
      "grid_import",
      bus="elec",
      carrier="grid",
      p_nom_extendable=True,
      p_nom_max=1e6,
      marginal_cost=grid_energy_cost,  # 0.35e3 eur/MWh
      capital_cost=grid_capacity_cost)  # 40e3 eur/MW annual

# Solar generators (rooftop)
for loc in solar_ts.columns:
    n.add("Generator",
          f"solar_{loc}",
          bus="elec",
          carrier="solar",
          p_max_pu=solar_ts[loc],
          capital_cost=data.at["PVRoof", "capex_annual"],
          p_nom_extendable=True,
          p_nom_max=solar_dict[loc])

# Solar freefield
n.add("Generator",
      "solar_S_Free",
      bus="elec",
      carrier="solar",
      p_max_pu=solar_ts["S"],
      capital_cost=data.at["PVFree", "capex_annual"],
      p_nom_extendable=True,
      p_nom_max=840)

# Wind
n.add("Generator",
      "wind",
      bus="elec",
      carrier="wind",
      p_max_pu=wind_ts["CF"],
      capital_cost=data.at["onwind", "capex_annual"],
      p_nom_extendable=True,)
      #p_nom_max=2000) # own estimate

# --------- Links ----------
# CHP: gas -> electricity (40%) + heat (50%)
n.add("Link",
      "CHP",
      bus0="gas",
      bus1="elec",
      bus2="heat",
      p_nom=1.5875,  # 0.635 MW_el / 0.4
      efficiency=0.40,
      efficiency2=0.50,
      capital_cost=data.at["CHP", "capex_annual"],  # eur/MW_th/year
      marginal_cost=data.at["CHP", "opex"])  # 0.17e3 eur/MWh biogas cost

# heatsink (dump heat)
n.add("Link",
      "heat_dump",
      bus0="heat",
      bus1="",  # Empty string means heat is discarded?
      p_nom=1e6,  # )unlimited)
      efficiency=1.0,
      marginal_cost=0)  # Free to dump

# --------- Storage ----------
# Flow storage (battery) - using Store + Links
# Store: holds energy (€/MWh cost)
# Links: charge and discharge (€/kW cost)
# Create a battery bus to connect store and links
n.add("Bus", "battery")

# Add the Store (energy capacity)
n.add("Store",
      "flow_storage",
      bus="battery",
      e_nom=0.72,  # Energy capacity in MWh
      e_cyclic=True,  # State of charge is cyclic (end = start)
      capital_cost= data.at["flow_storage", "capex_annual"])  # €/MWh/year

# Add charging Link (grid -> battery)
n.add("Link",
      "flow_storage_charge",
      bus0="elec",
      bus1="battery",
      p_nom=0.12,  # Charging power in MW
      efficiency=0.9,  # Charging efficiency
      capital_cost=0)  # €/MW/year for charging

# Add discharging Link (battery -> grid)
n.add("Link",
      "flow_storage_discharge",
      bus0="battery",
      bus1="elec",
      p_nom=0.12,  # Discharging power in MW
      efficiency=0.9,  # Discharging efficiency
      capital_cost=0)  # €/MW/year for discharging

# Print cost summary
print("\n=== Annualized Costs (€/MW/year or €/MWh) ===")
print(data[["capex", "FOM", "capex_annual", "opex"]])
print(f"\nGrid: {grid_capacity_cost} €/MW + {grid_energy_cost} €/MWh")
# print(f"\nFlow storage (Store+Links):")
# print(f"  - Energy: {flow_energy_capex} €/MWh/year for {flow_energy_capacity} MWh")
# print(f"  - Power: {flow_power_capex} €/MW/year for {flow_power_capacity} MW charge/discharge")

Index(['heat_dump'], dtype='object', name='name')
Index(['heat_dump'], dtype='object', name='name')
Index(['heat_dump'], dtype='object', name='name')



=== Annualized Costs (€/MW/year or €/MWh) ===
                     capex      FOM   capex_annual   opex
PVFree            400000.0  10000.0   40893.583860    0.0
PVRoof           1000000.0  12000.0   89233.959649    0.0
onwind           1100000.0  14000.0  105768.307420   50.0
CHP               400000.0  14000.0   51757.170297  170.0
flow_storage      500000.0  10000.0   57196.462872    0.0
thermal_storage        8.0      0.0       0.755143    0.0
grid                   0.0      0.0       0.000000  350.0

Grid: 40000.0 €/MW + 350.0 €/MWh


In [ ]:
n.optimize(solver_name='gurobi', keep_files=False)